In [1]:
import os
import json
import re
import tempfile
from pdf2image import convert_from_path
import fitz
from PIL import Image
import io
import pandas as pd
import time
from langchain_upstage import UpstageLayoutAnalysisLoader
from rainbow_html_transformer import HTMLToTextWithMarkdownTables
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# 텍스트 로드
file_path = './processed_txt/SHL0165_The안심VIP저축보험Ⅱ(무배당)_P116.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    document_text = file.read()

# # 텍스트 스플릿
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# chunks = text_splitter.create_documents([document_text])

# # 각 청킹 내용 출력
# for i, chunk in enumerate(chunks):
#     print(f"Chunk {i+1}:")
#     print(chunk.page_content)

In [3]:
# 목차를 인식하여 청킹 단위를 설정하는 함수
def extract_table_of_contents(text):
    # 목차 시작 패턴
    start_pattern = re.compile(r"목\s*차")
    # 목차 종료 패턴 ([별첨2] 이후의 첫 번째 빈 줄)
    end_pattern = re.compile(r"\[별첨2\].*?\n\s*\n", re.DOTALL)
    
    # 목차 추출
    start_match = start_pattern.search(text)
    if start_match:
        start_index = start_match.start()
        end_match = end_pattern.search(text[start_index:])
        if end_match:
            end_index = start_index + end_match.end()
            toc_text = text[start_index:end_index]
            
            # 목차 항목 추출
            toc_items = []
            lines = toc_text.split('\n')
            i = 0
            while i < len(lines):
                line = lines[i].strip()
                if line and not line.startswith("목 차") and not line.startswith("---"):
                    # 현재 줄에 페이지 번호가 있는 경우
                    match = re.match(r"(.+)\s+(\d+)$", line)
                    if match:
                        toc_items.append((match.group(1).strip(), match.group(2)))
                    elif i + 1 < len(lines) and lines[i+1].strip().isdigit():
                        # 다음 줄이 숫자만으로 이루어진 경우
                        toc_items.append((line, lines[i+1].strip()))
                        i += 1  # 다음 줄(페이지 번호)을 건너뛰기
                    elif not re.match(r"^--- End of Page \d+ ---$", line):
                        # "End of Page" 문구가 아닌 경우에만 추가
                        toc_items.append((line, ""))
                i += 1
            
            return toc_items
    
    return []

# 목차를 추출하고 결과 출력
toc = extract_table_of_contents(document_text)
for item, page in toc:
    print(f"목차: {item}, 페이지: {page}")

목차: 약관 가이드북, 페이지: 11
목차: 약관 요약서, 페이지: 15
목차: 주요보험용어 해설, 페이지: 23
목차: 가입부터 지급까지 쉽게 찾기!, 페이지: 25
목차: The안심VIP저축보험 II (무배당), 페이지: 29
목차: 제1관 목적 및 용어의 정의, 페이지: 29
목차: 제1 조 목적, 페이지: 29
목차: 제2조 용어의 정의, 페이지: 29
목차: 제2조의2 한국표준질병 ·사인분류 적용 기준, 페이지: 32
목차: 제2조의3 유지보너스에 관한 사항, 페이지: 33
목차: 제2관 보험금의 지급, 페이지: 34
목차: 제3조 보험금의 지급사유, 페이지: 34
목차: 제4조 보험금 지급에 관한 세부규정, 페이지: 34
목차: 제5조 보험금을 지급하지 않는 사유, 페이지: 35
목차: 제6조 보험금 지급사유의 발생통지, 페이지: 36
목차: 제7조 보험금의 청구, 페이지: 36
목차: 제8조 보험금의 지급절차, 페이지: 37
목차: 제9조 보험금 받는 방법의 변경, 페이지: 38
목차: 제10조 주소변경 통지, 페이지: 38
목차: 제11조 보험수익자의 지정, 페이지: 38
목차: 제12조 대표자의 지정, 페이지: 39
목차: 제3관 계약자의 계약 전 알릴 의무 등, 페이지: 40
목차: 제13조 계약 전 알릴 의무, 페이지: 40
목차: 제14조 계약 전 알릴 의무 위반의 효과, 페이지: 41
목차: 제15조 사기에 의한 계약, 페이지: 42
목차: 제4관 보험계약의 성립과 유지, 페이지: 42
목차: 제 16조 보험계약의 성립, 페이지: 42
목차: 제17조 청약의 철회, 페이지: 44
목차: 제18조 약관교부 및 설명의무 등, 페이지: 46
목차: 제19조 계약의 무효, 페이지: 47
목차: 제20조 계약내용의 변경 등, 페이지: 48
목차: 제20조의2 피보험자 변경에 관한 사항, 페이지: 49
목차: 제21조 보험나이 등, 페이지: 50
목차: 제22조 계약의 소멸, 페이지: 51
목차: 제5관 보험료의 납입, 페이지: 52
목차: 제23

In [4]:
# 텍스트 문서를 페이지 단위로 분리하고 페이지 번호와 페이지 텍스트 내용을 반환하는 함수
def split_pages(text):
    # 페이지 구분자를 기준으로 텍스트를 페이지 단위로 분리
    page_splits = re.split(r"(--- End of Page \d+ ---)", text)
    pages = []
    current_page = 1
    current_content = ""

    for split in page_splits:
        if split.startswith("--- End of Page"):
            # 페이지 번호 추출
            page_num = int(re.search(r"\d+", split).group())
            
            # 빈 페이지 처리
            while current_page < page_num:
                pages.append((current_page, ""))
                current_page += 1
            
            pages.append((current_page, current_content.strip()))
            current_page += 1
            current_content = ""
        else:
            current_content += split

    # 마지막 페이지 추가
    if current_content.strip() or current_page == len(pages) + 1:
        pages.append((current_page, current_content.strip()))

    return pages

pages = split_pages(document_text)
# 디버깅을 위해 페이지 번호와 내용 출력
for page_num, page_content in pages:
    print(f"Page {page_num}: {page_content[:100]}...")  # 첫 100자만 출력
    if page_num == 5:
        break

Page 1: 약관
The안심VIP저축보험�
(무배당)...
Page 2: 고객에게 드리는 감사의 인사말씀
먼저, 변함없는 신뢰와 성원을 보내주신 고객여러분께
진심으로 깊은 감사의 말씀을 드립니다.
2021년 7월 새롭게 태어난 신한라이프는
대한민국 생명...
Page 3: 고객 권리 안내문
금융서비스 이용범위
신한라이프는 고객님의 개인신용정보처리 동의목적 범위(금융거래설정/유지여부판단, 고객동의사항 등)내에서
최소한의 정보만 수집, 이용하고 있습니다...
Page 4: - 전 화 : 콜센터 1588-5580 / 080-550-5580
라. 신용정보의 열람 및 정정청구 (법 제38조)
- 고객님께서는 본인임을 확인받아 신한라이프가 보유하고 있는 고...
Page 5: 위의 권리사항 관련하여 불편함을 느끼시거나 애로가 있으신 경우,
아래의 담당자 앞으로 연락하여 주시기 바랍니다.
가. 신한라이프 신용정보관리·보호인
- 02-3455-4316(서울...


In [5]:
# 문서의 주제를 추출하는 함수
def longest_common_substring(s1, s2):
    m = [[0] * (1 + len(s2)) for _ in range(1 + len(s1))]
    longest, x_longest = 0, 0
    for x in range(1, 1 + len(s1)):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
    return s1[x_longest - longest: x_longest]

def extract_document_topic(first_page, toc):
    # 첫 페이지에서 "약관" 다음에 나오는 텍스트 추출
    first_page_pattern = re.compile(r"약관\s*(.*?)\s*\n", re.DOTALL)
    first_page_match = first_page_pattern.search(first_page)
    first_page_topic = first_page_match.group(1).strip() if first_page_match else ""
    # print("first_page_topic:", first_page_topic)

    # 목차에서 "가입부터 지급까지 ~"와 "제1관 ~" 사이의 목차명 추출
    toc_text = "\n".join([item for item, _ in toc])
    toc_pattern = re.compile(r"가입부터 지급까지.*?\n(.*?)(?=\n제1관)", re.DOTALL)
    toc_match = toc_pattern.search(toc_text)
    toc_topic = toc_match.group(1).strip() if toc_match else ""
    # print("toc_topic:", toc_topic)

    # 첫 페이지와 목차에서 추출한 내용 비교
    if first_page_topic and toc_topic:
        # 최장 공통 부분 문자열 찾기
        common_substring = longest_common_substring(first_page_topic, toc_topic)
        similarity = len(common_substring) / max(len(first_page_topic), len(toc_topic))
        # print("Longest common substring:", common_substring)
        # print("Similarity:", similarity)
        
        if similarity > 0.5:  # 유사도 임계값 (필요에 따라 조정)
            return toc_topic
        else:
            return first_page_topic
    elif toc_topic:
        return toc_topic
    elif first_page_topic:
        return first_page_topic
    else:
        return "Unknown Topic"

document_topic = extract_document_topic(pages[0][1], toc)
print("document_topic :",document_topic)

document_topic : The안심VIP저축보험 II (무배당)


In [6]:

# 문서의 구분을 추출하는 함수
def process_document_classifications(toc, pages):
    """
    문서 목차와 페이지로 구분된 문서를 입력받아 문서의 구분과 시작 페이지를 추출하는 함수
    """
    sections = []
    toc_dict = {item.lower(): page for item, page in toc}  # 대소문자 구분 없이 비교하기 위해 소문자로 변환
    
    # "문서개요" 섹션 찾기
    sections.append(("문서개요", "1"))
    
    # 나머지 섹션 찾기
    section_order = [
        "약관 가이드북",
        "약관 요약서",
        "주요보험용어 해설",
        "가입부터 지급까지 쉽게 찾기",
        "주계약사항",
        "특약사항",
        "별첨"
    ]
    
    document_topic = extract_document_topic(pages[0][1], toc)
    
    for section in section_order:
        section_lower = section.lower()
        if section_lower in toc_dict:
            sections.append((section, toc_dict[section_lower]))
        elif any(section_lower in item.lower() for item, _ in toc):
            # 부분 일치하는 항목 찾기
            matching_item = next(item for item, _ in toc if section_lower in item.lower())
            sections.append((section, toc_dict[matching_item.lower()]))
        elif section == "주계약사항":
            # 주계약사항은 문서 주제와 일치하는 항목을 찾습니다
            if document_topic.lower() in toc_dict:
                sections.append((section, toc_dict[document_topic.lower()]))
        elif section == "특약사항":
            # 특약사항은 "특약"이 포함된 모든 항목을 찾습니다
            special_items = [(item, page) for item, page in toc if "특약" in item.lower()]
            if special_items:
                sections.append((section, special_items[0][1]))  # 첫 번째 특약의 페이지 번호 사용
        elif section == "별첨":
            # 별첨은 "[별첨"으로 시작하는 항목을 찾습니다
            attachment_items = [(item, page) for item, page in toc if item.lower().startswith("[별첨")]
            if attachment_items:
                sections.append((section, attachment_items[0][1]))
    
    return sections

# 사용 예시
classifications = process_document_classifications(toc, pages)
print("문서 구분과 시작 페이지:")
for doc_class, start_page in classifications:
    print(f"({doc_class}, {start_page})")

문서 구분과 시작 페이지:
(문서개요, 1)
(약관 가이드북, 11)
(약관 요약서, 15)
(주요보험용어 해설, 23)
(가입부터 지급까지 쉽게 찾기, 25)
(주계약사항, 29)
(특약사항, 65)
(별첨, 87)


In [7]:
# 해당 문서구분을 청킹하는 함수
def chunk_by_doc_class(doc_class, classifications, pages):
    """
    문서구분 명칭과 문서구분 리스트와 전체 텍스트 문서를 입력으로 받아,
    전체 텍스트 문서에서 해당 문서구분 명칭 부분만을 추출하여 반환하는 함수
    doc_class : 문서구분 명칭
    classifications : 문서구분 리스트
    pages : 전체 텍스트 문서
    """
    # 페이지 순으로 정렬
    classifications.sort(key=lambda x: int(x[1]))
    
    start_page = None
    end_page = None
    
    # 해당 문서구분의 시작 페이지와 다음순번 문서구분의 시작 페이지 찾기
    for i, (section, page) in enumerate(classifications):
        if section == doc_class:
            start_page = int(page)
            if i + 1 < len(classifications):
                end_page = int(classifications[i+1][1]) - 1
            break
    
    # 해당 문서구분이 없거나 마지막 문서구분인 경우 처리
    if start_page is None:
        return ""
    if end_page is None:
        end_page = len(pages)
    
    # 해당 문서구분 내용 추출
    doc_class_content = ""
    for page_num, content in pages:
        if start_page <= int(page_num) <= end_page:
            doc_class_content += content + "\n"
    
    return doc_class_content.strip()

# 사용 예시
doc_class = "주계약사항"
doc_class_content = chunk_by_doc_class(doc_class, classifications, pages)
print(f"{doc_class}:")

# 첫 10줄과 마지막 10줄 출력
lines = doc_class_content.split('\n')
first_10_lines = lines[:10]
last_10_lines = lines[-10:]

print("첫 10줄:")
print('\n'.join(first_10_lines))
print("\n...\n")  # 중간 생략 표시
print("마지막 10줄:")
print('\n'.join(last_10_lines))

주계약사항:
첫 10줄:
The안심VIP저축보험 11 (무배당)
제1관 목적 및 용어의 정의
제1조 목적
이 보험계약(이하 "계약"이라 합니다)은 보험계약자(이하 "계약자"라 합니다)와
보험회사(이하 "회사"라 합니다) 사이에 제3조(보험금의 지급사유)에 해당하는
피보험자의 위험을 보장하기 위하여 체결됩니다.
제2조 용어의 정의
이 계약에서 사용되는 용어의 정의는, 이 계약의 다른 조항에서 달리 정의되지
않는 한 다음과 같습니다.
1. 계약관계 관련 용어

...

마지막 10줄:
않습니다.
3. 계약자 등의 책임 있는 사유로 보험금 지급이 지연된 때에는 그 해당기간에 대한 이자는
지급되지 않을 수 있습니다. 다만, 회사는 계약자 등이 분쟁조정을 신청했다는 사유만으
로 이자지급을 거절하지 않습니다.
4. 가산이율 적용시 제8조(보험금의 지급절차) 제3항 각 호의 어느 하나에 해당되는 사유로
지연된 경우에는 해당기간에 대하여 가산이율을 적용하지 않습니다.
5. 가산이율 적용시 금융위원회 또는 금융감독원이 정당한 사유로 인정하는 경우에는 해당
기간에 대하여 가산이율을 적용하지 않습니다.
부표3 재해분류표
별첨2 [표 1] 참조


In [8]:
# InsuranceDocumentProcessor.py를 이용 보험약관을 청킹 한다.
from InsuranceDocumentProcessor import InsuranceDocumentProcessor

# InsuranceDocumentProcessor 인스턴스 생성
processor = InsuranceDocumentProcessor()

# 문서 처리 후 판다스 데이터프레임으로 반환
df = processor.process_document_to_dataframe(doc_class_content)
# print(df)
# 현재 날짜와 시간을 파일명에 포함
from datetime import datetime
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
excel_filename = f"insurance_document_chunks_{current_time}.xlsx"

# 엑셀 파일로 저장
output_path = os.path.join(os.getcwd(), excel_filename)
df.to_excel(output_path, index=False, engine='openpyxl')

print(f"데이터프레임이 다음 경로에 엑셀 파일로 저장되었습니다: {output_path}")
print(f"저장된 데이터의 행 수: {len(df)}")
print(f"저장된 데이터의 열: {', '.join(df.columns)}")

데이터프레임이 다음 경로에 엑셀 파일로 저장되었습니다: /home/samuel/Dev/RAG/AutoRAG-tutorial-ko/insurance_document_chunks_20240829_174425.xlsx
저장된 데이터의 행 수: 42
저장된 데이터의 열: 분류, 세분류, 청킹내용
